In [1]:
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine, MetaData, Table, Column, ForeignKey
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, ForeignKey, String, Integer, Float, DateTime, select, Index
from sqlalchemy import func
import numpy as np
import logging
logging.basicConfig()
logging.getLogger('sqlalchemy.engine').setLevel(logging.INFO)



In [2]:
def load_csv_to_db(conn,csv_filename,table_name):
    df = pd.read_csv(csv_filename)
    df.to_sql(table_name, conn, if_exists='append', index = False)

In [3]:
# Methods to handle database
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

In [26]:
metadata_obj = MetaData()
Ratings = Table(
    "Ratings",
    metadata_obj,
    Column("ratingId", Integer, primary_key=True, index=True),
    Column("userId", Integer),
    Column("movieId",Integer, ForeignKey("Movies.movieId")),
    Column("rating", Float),
    Column("timestamp", DateTime),
)

Movies = Table(
    "Movies",
    metadata_obj,
    Column("movieId", Integer, primary_key=True, index=True),
    Column("title", String),
    Column("genres", String)
)
temp_table = Table("Rating_table", metadata_obj,
                    Column("movieId", Integer, ForeignKey("Movies.movieId")),
                    Column("average_rating", Float),
                    prefixes=['TEMPORARY'],
                        )
engine = create_engine("sqlite:///Database/movielens.db")
metadata_obj.create_all(bind=engine)
conn = engine.connect()

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("Ratings")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("Movies")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("Rating_table")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:COMMIT


In [5]:
from sqlalchemy.types import Integer, String, DateTime
connection=create_connection("Database/movielens.db")
#load_csv_to_db(connection, "movielens/tags.csv", "Tags")
load_csv_to_db(connection, "movielens/movies.csv","Movies")
load_csv_to_db(connection, "movielens/ratings.csv", "Ratings")

2.6.0


In [36]:
def calculate_rating(sum_of_ratings_i, num_of_ratings_i, total_num_of_ratings, num_of_movies):
    avg_number_of_ratings = total_num_of_ratings/ num_of_movies
    if num_of_ratings_i<1:
        return 0
    penalty=(num_of_ratings_i-avg_number_of_ratings)/(avg_number_of_ratings)
    penalty = np.clip(penalty, -1, 0)
    return (sum_of_ratings_i/num_of_ratings_i)  + penalty 

In [37]:
#def create_rating_table():
query = select(Ratings.columns.movieId, func.count(Ratings.columns.ratingId), func.sum(Ratings.columns.rating)).group_by(Ratings.columns.movieId)#.with_hint(Ratings, "index(%(name)s ix_Ratings_ratingId)")
output = conn.execute(query)
results= output.fetchall()
df=pd.DataFrame(results)

INFO:sqlalchemy.engine.Engine:SELECT "Ratings"."movieId", count("Ratings"."ratingId") AS count_1, sum("Ratings".rating) AS sum_1 
FROM "Ratings" GROUP BY "Ratings"."movieId"
INFO:sqlalchemy.engine.Engine:[generated in 0.00067s] ()


In [38]:
num_of_ratings = np.array(df["count_1"])
sum_of_ratings = np.array(df["sum_1"])
movie_Ids = df["movieId"]
total_num_of_ratings = num_of_ratings.sum()
num_of_movies = sum_of_ratings.shape[0]
avg_of_rating = np.zeros(num_of_movies)

for i in range (num_of_movies):
    avg_of_rating[i] = calculate_rating(sum_of_ratings[i], num_of_ratings[i], total_num_of_ratings, num_of_movies)
avg_of_rating = np.round(avg_of_rating,2)
avg_of_rating = pd.Series(avg_of_rating)

In [39]:
cached_rating_table= pd.DataFrame(columns=["movieId", "average_rating"])
cached_rating_table["movieId"]= movie_Ids
cached_rating_table["average_rating"] = avg_of_rating
cached_rating_table.to_sql("Rating_table", conn, if_exists='append', index = False)


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("Rating_table")
INFO:sqlalchemy.engine.Engine:[raw sql] ()
INFO:sqlalchemy.engine.Engine:INSERT INTO "Rating_table" ("movieId", average_rating) VALUES (?, ?)
INFO:sqlalchemy.engine.Engine:[generated in 0.16066s] ((1, 3.89), (2, 3.25), (3, 3.14), (4, 2.85), (5, 3.06), (6, 3.85), (7, 3.36), (8, 3.11)  ... displaying 10 of 59047 total bound parameter sets ...  (209169, 2.0), (209171, 2.0))
INFO:sqlalchemy.engine.Engine:COMMIT
INFO:sqlalchemy.engine.Engine:SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
INFO:sqlalchemy.engine.Engine:[raw sql] ()


59047

In [40]:
def get_Movies():
    query = Movies.select()
    output = conn.execute(query)
    results = output.fetchall()
    return pd.DataFrame(results)
get_Movies()

INFO:sqlalchemy.engine.Engine:SELECT "Movies"."movieId", "Movies".title, "Movies".genres 
FROM "Movies"
INFO:sqlalchemy.engine.Engine:[generated in 0.00156s] ()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [10]:
#Full text search of movies: https://amitosh.medium.com/full-text-search-fts-with-postgresql-and-sqlalchemy-edc436330a0c
def search_Movies(keyword):
    query = select(Movies).where(Movies.columns.title.contains(keyword))
    output = conn.execute(query)
    results = output.fetchall()
    return pd.DataFrame(results)
search_Movies("toy")

INFO:sqlalchemy.engine.Engine:SELECT "Movies"."movieId", "Movies".title, "Movies".genres 
FROM "Movies" 
WHERE ("Movies".title LIKE '%' || ? || '%')
INFO:sqlalchemy.engine.Engine:[generated in 0.00087s] ('toy',)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2017,Babes in Toyland (1961),Children|Fantasy|Musical
2,2253,Toys (1992),Comedy|Fantasy
3,2480,Dry Cleaning (Nettoyage à sec) (1997),Drama
4,3086,Babes in Toyland (1934),Children|Comedy|Fantasy|Musical
5,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy
6,4929,"Toy, The (1982)",Comedy
7,5843,Toy Soldiers (1991),Action|Drama
8,26033,Giants and Toys (Kyojin to gangu) (1958),Comedy|Drama
9,78062,Puppet Master vs. Demonic Toys (Puppet Master ...,Comedy|Fantasy|Horror|Sci-Fi|Thriller


In [49]:

def Top10_rated_movies(genre):
    query = select([Movies.columns.title, Movies.columns.genres, temp_table.columns.average_rating]).select_from(Movies.join(temp_table,Movies.columns.movieId == temp_table.columns.movieId)).where(Movies.columns.genres.contains(genre)).order_by(temp_table.columns.average_rating.desc()).limit(10)
    output=conn.execute(query)
    results = output.fetchall()
    return pd.DataFrame(results)
Top10_rated_movies("Crime")


INFO:sqlalchemy.engine.Engine:SELECT "Movies".title, "Movies".genres, "Rating_table".average_rating 
FROM "Movies" JOIN "Rating_table" ON "Movies"."movieId" = "Rating_table"."movieId" 
WHERE ("Movies".genres LIKE '%' || ? || '%') ORDER BY "Rating_table".average_rating DESC
 LIMIT ? OFFSET ?
INFO:sqlalchemy.engine.Engine:[cached since 160.1s ago] ('Crime', 10, 0)


,title,genres,average_rating
0,"Shawshank Redemption, The (1994)",Crime|Drama,4.41
1,"Shawshank Redemption, The (1994)",Crime|Drama,4.40
2,"Godfather, The (1972)",Crime|Drama,4.32
3,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,4.30
4,"Godfather, The (1972)",Crime|Drama,4.30
5,"Godfather: Part II, The (1974)",Crime|Drama,4.30
6,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,4.28
7,"Godfather: Part II, The (1974)",Crime|Drama,4.26
8,Fight Club (1999),Action|Crime|Drama|Thriller,4.23
9,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.20


In [262]:
def find_similar_users(user_id,lim_movies):
    #get all movies that were rated by the given user
    all_rated_movies_by_user_sql=conn.execute(select(Ratings.columns.movieId, Ratings.columns.rating).where(Ratings.columns.userId== user_id).with_hint(Ratings, "index(%(name)s ix_Ratings_ratingId)").limit(lim_movies)).fetchall()
    all_rated_movies_by_user_df =pd.DataFrame(all_rated_movies_by_user_sql)
    all_rated_movies_by_user_np = all_rated_movies_by_user_df.to_numpy()

    #extract the given movie Ids and ratings by the given user
    movie_Ids = all_rated_movies_by_user_df["movieId"].to_list()
    ratings = all_rated_movies_by_user_df["rating"]

    #query for all ratings in the Database that were given to these movies
    all_ratings_sql = conn.execute(select(Ratings.columns.userId,Ratings.columns.movieId, Ratings.columns.rating).where(Ratings.columns.movieId.in_(movie_Ids), Ratings.columns.userId.is_not(user_id))).fetchall()
    all_ratings_np=pd.DataFrame(all_ratings_sql).to_numpy()

    #get the highest user id for creating a hit list for each user
    max_user = int(all_ratings_np[:,0].max())
    user_hits = np.zeros(max_user+1, dtype=np.uint8)
    idx = 0

    #iterate through all ratings and update the hit list accordingly, the userid is the index of the hitlist
    for row in all_ratings_np:
        user_id = int(row[0]); movie_id = int(row[1]); rating = row[2]
        if rating == all_rated_movies_by_user_np[np.where(all_rated_movies_by_user_np[:,0]== movie_id),1][0][0]:
            user_hits[user_id] = user_hits[user_id] + 1

    
    nr_users = int(np.count_nonzero(user_hits))
    user_hit_list_np = np.zeros((nr_users-1,2))
    idx_user=0
    #only keep user_ids that had atleast one hit
    for i in range(max_user):
        if user_hits[i]>0:
            user_hit_list_np[idx_user,0]=i
            user_hit_list_np[idx_user,1]= user_hits[i]
            idx_user = idx_user + 1
    #convert back to dataframe
    user_hit_list_df = pd.DataFrame(columns=["userId", "similar_rated_movies"], data=user_hit_list_np).sort_values(by="similar_rated_movies",ascending=False)
    return(user_hit_list_df)
find_similar_users(4540, lim_movies=10)

INFO:sqlalchemy.engine.Engine:SELECT "Ratings"."movieId", "Ratings".rating 
FROM "Ratings" 
WHERE "Ratings"."userId" = ?
 LIMIT ? OFFSET ?
INFO:sqlalchemy.engine.Engine:[cached since 282.3s ago] (4540, 10, 0)
INFO:sqlalchemy.engine.Engine:SELECT "Ratings"."userId", "Ratings"."movieId", "Ratings".rating 
FROM "Ratings" 
WHERE "Ratings"."movieId" IN (?, ?, ?, ?, ?, ?, ?, ?, ?, ?) AND "Ratings"."userId" IS NOT ?
INFO:sqlalchemy.engine.Engine:[cached since 1637s ago] (7, 11, 17, 19, 21, 26, 39, 41, 64, 74, 4540)


,userId,similar_rated_movies
15235,93261.0,7.0
3233,20162.0,6.0
21749,133921.0,6.0
19235,117974.0,6.0
6479,39848.0,6.0
...,...,...
9979,61063.0,1.0
9978,61059.0,1.0
9976,61053.0,1.0
9975,61032.0,1.0


In [70]:
str(session.query(Movies.movieId, func.count(Ratings.rating), func.sum(Ratings.rating)).join(Ratings, Ratings.movieId == Movies.movieId, isouter=True).with_hint(Movies, "WITH INDEX movieId"))

'SELECT "Movies"."movieId" AS "Movies_movieId", count("Ratings".rating) AS count_1, sum("Ratings".rating) AS sum_1 \nFROM "Movies" LEFT OUTER JOIN "Ratings" ON "Ratings"."movieId" = "Movies"."movieId"'

In [80]:
conn.execute("EXPLAIN QUERY PLAN SELECT Movies.movieId FROM Movies")


In [19]:
def best_rated_movies(genre):
    d
s

In [20]:
best_rated_movies("Comedy")